# Comunicação segura com TAES (Tweakable AES)

## Introdução
O uso de *tweakable ciphers* surge da necessidade de garantir duas propriedades, confidencialidade e autenticação, que surgem muitas vezes simultaneamente quando se pretende estabelecer uma comunicação segura entre dois partidos.

Estas primitivas recorrem ao conceito de *tweak* que corresponde a um valor que tem por como objetivo introduzir
variabilidade no processo de cifragem, mesmo quando a chave usada para cifrar uma dada mensagem é a mesma.
É importante salientar a diferença entre estes dois conceitos *tweak* e chave, sendo que o segundo é custoso de alterar visto que condiciona diretamente a a permutação efetuada por uma dada cifra enquanto que o primeiro
é simples de alterar, diferindo mesmo entre blocos diferentes de texto limpo cifrado.
Adicionalmente o uso de um *tweak* é facilmente integrável nos diferentes modos de funcionamento de cifras por blocos, sendo esta umas das suas vantagens.

Tendo isto em conta a presente implementação apresenta uma proposta para um protcolo de comunicação segura **i.e.** confidencial e autenticada, entre um Emissor (*Emitter*) e um Recetor(*Receiver*), recorrendo para isso à construção 
TAE (*Tweakable Authenticated Encryption*), semelhante ao modo *OCB*, e à cifra AES.

## Connection

In [44]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        genECDSAKeyPair("emitter")
        genECDSAKeyPair("receiver")
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Primitivas criptográficas

As primitivas criptográficas de seguida apresentadas implementam não só funções referentes ao acordo de chaves *Diffie-Hellman* mas também à cifragem/decifragem da informação trocada entre dois as duas partes.

### Chaves de longa duração
Analogamente à versão [DH_DSA](DH_DSA.ipynb) a autenticação do protocolo de troca de chaves *Diffie-Hellman* é garantida recorrendo a um esquema de assinaturas digitais no entanto, neste caso são usadas curvas elítpicas como alternativa ao algoritmo **DSA**, resultando num esquema designado **ECDSA** que também visa garantir que o acordo de chaves não é vulnerável a ataques *Man-in-The-Middle*. Também o protocolo de troca de chaves recorre, *Diffie-Hellman* é apresentado na sua variante com recurso a curvas elípticas designando-se por **ECDH**.

O uso de curvas elítpicas apresenta diversas vantagens face à implementação com aritmética modular,  permitindo obter uma maior *performance* e ainda garantir níveis de segurança semelhantes com chaves de dimensões inferiores.

A geração de chaves de longa duração é implementada pela função `genECDSAKeyPair` que gera um par de chaves para assinaturas ECDSA e guarda cada uma em ficheiros **PEM**.

A chave privada é armazenada com recurso ao formato **PKCS8** e cada par de chaves é gerado apenas na criação da conexão **i.e.** na chamada do construtor da classe `Connection`, mantendo-se constante para ligações estabelecidas na mesma instância da classe.

In [45]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec

def genECDSAKeyPair(fname):
    privKeyF = open(fname+"_priv.pem", 'wb')
    pubKeyF = open(fname+"_pub.pem", 'wb')
   
    privKey = ec.generate_private_key(
         ec.SECP256R1(),
         default_backend()
    )
    pubKey = privKey.public_key()
    
    privKeyDump = privKey.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(fname.encode("ascii")) # Store encrypted key
    )
    pubKeyDump = pubKey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    privKeyF.write(privKeyDump)
    privKeyF.close()
    pubKeyF.write(pubKeyDump)
    pubKeyF.close()

As funções que se seguem: `verify_tag`, `pad_message`, `parity_block`,`create_block_hash`,`xor_bytes`,`encrypt_message` e `decipher_message`; estão respetivamente descritas e servem de funções auxiliares durante o processo de comunicação.

Note-se primeiramente que o tempo de geração dos parâmetros do protocolo de acordo de chaves Diffie-Hellman (P e G), recorrendo ao método `generate_parameters()`, é considerável, tendo-se optado por fixar os mesmos, como se constata de seguida.

Todas estas as funções auxiliares têm como referência a construção da cifra `TAES` (AES  na  sua versão *tweak*) e têm em consideração o seguinte esquema:

![title](1.png)

Das funções auxiliares já referidas existe necessidade de esclarecer, previamente, o funcionamento mais detalhado de algumas delas, nomeadamente as funções `encrypt_message`, `decipher_message` e `create_block_hash`.

A função `encrypt_message` começa por efetuar o *padding* da mensagem inicial, recorrendo à função `pad_message`,de modo a que a mesma tenha um comprimento múltiplo do tamanho de cada bloco (128 bits para **AES-ECB**), resultante da divisão da mensagem.

De seguida todos os blocos, com exceção do último, são combinados com o *hash* do *tweak* correspondente, que é calculado com a seguinte expressão: $$Z_{i} = \nu \ || \ i \ || \ 0,\  tendo \ que \ i \in \mathbb{N}.$$
O valor $\nu$ deve ser um *nonce* e, quando isto é assegurado, o esquema apresenta segurança perfeita.
Na presente implementação este valor (`tweak_iv`) é obtido da seguinte maneira `tweak = str(time.time()).encode("ascii")[-4:]+os.urandom(4)` combinando 4 *bytes* do tempo Unix no instante atual com um valor aleatório de 4 *bytes*.

O último bloco da mensagem é processado de maneira diferente. A cifra recebe como parâmetro o comprimento do último bloco antes de ter sido efetuado o *padding*, $len = |M_{m}|$ e o respetivo *tweak* combinando o criptograma resultante, com recurso a uma operação XOR, com a mensagem $M_{m} || 0^*$.

Por fim, calcula-se a *tag* da mensagem, responsável pela sua autenticação. Esta *tag* é obtida fazendo a cifragem da *parity/checksum* com o *tweak* $$Z_{0} = \nu \ ||  \ \ |p|\ \  || \ 0.$$ Note-se que:

$$ Parity = M_{1} \oplus M_{2} \oplus M_{3} ... \oplus M_{m}$$

Consequentemente, a função `decipher_message` corresponde ao processo inverso da função anteriormente descrita.

A função `create_block_hash` é responsável por calcular o *hash* do *tweak* para cada bloco, que é combinado com o respetivo bloco da mensagem/criptograma, tendo em conta a seguinte expressão: 

$$ ctxt = E_k(ptxt \oplus h(tweak)) \oplus h(tweak)$$

onde:
- $ctxt$ : criptograma gerado
- $ptxt$ : mensagem original
- $tweak$: tweak/nonce

Observe-se, tendo em conta que $ D_k = E^{-1}_k $ ,que a obtenção do texto limpo original ($ptxt$) pode ser conseguida da seguinte maneira:

$$ ptxt = D_k(ctxt \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak)) \oplus h(tweak) \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak))) \oplus h(tweak) \rightarrow ptxt \oplus h(tweak) \oplus h(tweak) \rightarrow ptxt $$




In [46]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import os

def sign_message(priv_key, msg):
    """ Signs a particular message with a given private key
    """
    signature = priv_key.sign(msg,ec.ECDSA(hashes.SHA256()))
    return {'sig': signature, 'msg' : msg}

def verify_signature(pub_key, signature, msg):
    """ Verify that the private key associated with a particular public key was used to sign that particular message
    """
    try:
        pub_key.verify(signature,msg,ec.ECDSA(hashes.SHA256()))
        return True
    except:
        return False


def verify_tag(key, tweak_iv, tag, bs, ctxt_len, ctxt_parity):
    """ Verify cryptogram authentication
    """
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).decryptor()
    lower_parity = list((ctxt_len*2+1).to_bytes(bs//2, 'big'))      
    block_tweak = create_block_hash(tweak_iv, lower_parity)
    tag_out = xor_bytes(tag, block_tweak, 0, bs)
    parity = xor_bytes(cipher.update(tag_out) + cipher.finalize(), block_tweak, 0, bs)
    
    return (ctxt_parity == parity)

def decipher_message(key, tweak_iv, ctxt, tag):
    """ Extracts the plain text from a cryptogram resulting from a tweakable cipher
        where ptxt = dec(ctxt ^ hash(tweak)) ^ hash(tweak)
    """
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).decryptor()
    tweak_iv = list(tweak_iv)
    bs = len(tweak_iv)*2
    parity = bytes([0]*bs)
    ptxt = bytes()
    pad_ctxt = pad_message(ctxt, bs)
    last_block_len = bs - (len(pad_ctxt) - len(ctxt))
    last_block_len_b = last_block_len.to_bytes(bs, 'big') #length of last cipher text block
    
    block_index = 0
    offset = 0
    block_count = (len(ctxt)-bs) // bs
    
    while block_index < block_count:
        lower_tweak = list((block_index*2).to_bytes(bs//2, 'big'))         # convert i*2 to binary format
        block_tweak = create_block_hash(tweak_iv, lower_tweak)             # tweak digest
        block_in = xor_bytes(pad_ctxt, block_tweak, offset, bs)
        ptxt_block = xor_bytes(cipher.update(block_in), block_tweak, 0, bs)
        ptxt += ptxt_block
        parity = xor_bytes(ptxt_block, parity, 0, bs)                      # update parity
        block_index += 1
        offset = block_index * 16
    
    ptxt += cipher.finalize()
    
    # Decipher last block
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    lower_tweak = list((block_index*2).to_bytes(bs//2, 'big'))         # convert i*2 to binary format
    block_tweak = create_block_hash(tweak_iv, lower_tweak)             # tweak digest
    block_in = xor_bytes(last_block_len_b, block_tweak, 0, bs)
    block_out = xor_bytes(cipher.update(block_in) + cipher.finalize(), block_tweak, 0, bs)
    last_block = xor_bytes(ctxt, block_out, offset, last_block_len) + bytes(bs-last_block_len)
    parity = xor_bytes(last_block, parity, 0, bs)
    ptxt += last_block
    
    # verify cryptogram tag
    if(verify_tag(key, tweak_iv, tag, bs, len(pad_ctxt)*8, parity)):
        return ptxt
    else: 
       return None
    
def pad_message(msg, block_size):
    if (len(msg) % block_size) != 0:
        pad_size = block_size - (len(msg) % block_size)
        msg = msg + bytes(pad_size)
        
    return msg
    
def parity_block(tweak_iv, msg_len, bs, parity):
    lower_parity = list((msg_len * 16 + 1).to_bytes(bs//2, 'big'))
    block_tweak = create_block_hash(tweak_iv, lower_parity)
    tag_in = xor_bytes(parity, block_tweak, 0, bs)
    return (block_tweak, tag_in)

def create_block_hash(tweak_iv, lower_tweak):
    tweak_iv.extend(lower_tweak)
    block_tweak = bytes(tweak_iv)
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(block_tweak)
    return (digest.finalize())[16:]
    
def encrypt_message(msg, tweak_iv, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    tweak_iv_l = list(tweak_iv)
    bs = len(tweak_iv)*2
    pad_msg = pad_message(msg, bs)
    last_block_len = bs - (len(pad_msg) - len(msg))
    last_block_len_b = last_block_len.to_bytes(bs, 'big') #length of last cipher text block
    parity = bytes([0]*bs)
    ctxt = bytes()
    
    block_index = 0
    offset = 0
    block_count = (len(pad_msg)-bs) // bs
    
    while block_index < block_count:
        parity = xor_bytes(pad_msg, parity, offset, bs)                         # update parity
        lower_tweak = list((block_index*2).to_bytes(bs//2, 'big'))              # convert i*2 to binary format
        block_tweak = create_block_hash(tweak_iv_l, lower_tweak)                # tweak digest
        block_in = xor_bytes(pad_msg, block_tweak, offset, bs)
        ctxt_block = xor_bytes(cipher.update(block_in), block_tweak, 0, bs)
        ctxt += ctxt_block
        block_index += 1
        offset = block_index * 16
    
    parity = xor_bytes(pad_msg, parity, offset, bs)
    lower_tweak = list((block_index*2).to_bytes(bs//2, 'big'))
    block_tweak = create_block_hash(tweak_iv_l, lower_tweak)           # tweak digest
    block_in = xor_bytes(last_block_len_b, block_tweak, 0, bs)           # xor last block length with tweak
    ctxt_block = xor_bytes(cipher.update(block_in), block_tweak, 0, bs)
    ctxt_block = xor_bytes(pad_msg, ctxt_block, offset, bs)[:last_block_len]
    ctxt += (ctxt_block + cipher.finalize())
    
    # Compute authentication tag
    block_tag = parity_block(tweak_iv_l, len(pad_msg), bs, parity)
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    tag = xor_bytes(cipher.update(block_tag[1]) + cipher.finalize(), block_tag[0], 0, bs)
    
    payload = {'content': ctxt, 'tag': tag, 'tweak_iv': tweak_iv}
    
    return payload
    
    
def xor_bytes(m1, m2, offset, bs):
    """ Xor bytes
    """
    xored_res = []
    for i in range(0, bs):
        xored_res.append(m1[offset+i] ^ m2[i])
    return bytes(xored_res)

## Emitter

O Emitter começa por gerar a chave partilhada para cifrar as mensagens trocadas com o Receiver, conforme o protocolo de acordo de chaves *ECDH*, e adoptando a curva *standard NIST P-521*, armazenando-a numa variável local. Adicionalmente, e para prevenir a realização de ataques Man-In-The-Middle, que violam a autenticidade das mensagens trocadas durante o protocolo, este recorre a chaves para assinar e verificar as assinaturas ECDSA, que estavam armazenadas em ficheiros com nomes alusivos a cada uma dessas operações e que têm como referência a a curva *standard NIST P-256*.

Com recurso às funções `sign_message` e `verify_signature` efetua-se nomeadamente a assinatura e a respetiva verificação da mesma. Sendo que a primeira recebe como parâmetros a chave privada usada para assinar (`sign_key`) e a mensagem `public_key_pem` e a segunda recebe a chave pública (`ver_key`), a assinatura esperada (`sig`) e a mensagem à qual esta assinatura se refere (`msg`).

Se durante no protocolo de troca de chaves não ocorrer nenhuma falha ao nível da verificação da troca das mesmas, sinalizada pela mensagem b"0", determina-se a chave acordada no  protocolo DH (`s_key`). Posteriormente, recorrendo a esta,`s_key`, e à função auxiliar `deriveKey` gera-se a chave da cifra (`c_key`) e a chave do HMAC (`h_key`).

Seguidamente gerou-se o `tweak`(*nonce*)como uma combinação do tempo no instante atual, em segundos, com 8 *bytes* aleatórios.

Posteriormente calcula-se o hash do valor do tweak (`tweak_h`) e ainda o HMAC de `header + content_new + iv + tweak_h` que é armazenado no campo `m_hmac`. Tal permite que a autenticação seja feita via uma *tag* que deve ser utilizada no momento de decifragem dos dados e, caso não corresponda ao valor esperado, resulta numa exceção.

Para além da mensagem trocada entre os dois interveniente são ainda enviados dados que permitem a sua decifragem
por parte do `Receiver` nomeadamente a *tag* esperada para decifrar os mesmos, o *hash* do tweak utilizado e ainda os metadados previamente autenticados:

payload = {'header': header, 'content': content_new, 'm_hmac': m_hmac, 'tweak_h': tweak_h}

In [47]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
import os, time


def Emitter(conn):
    try:
        # Load long-term keys
        sign_key_dump = open("emitter_priv.pem", "rb").read()
        sign_key = load_pem_private_key(sign_key_dump, "emitter".encode("ascii"), backend=default_backend())
        
        ver_key_dump = open("receiver_pub.pem", "rb").read()
        ver_key = load_pem_public_key(ver_key_dump, backend=default_backend()) 
       
        # Session keys
        priv_key = ec.generate_private_key(ec.SECP521R1(), default_backend())
        public_key_pem = priv_key.public_key().public_bytes(
                                        encoding=serialization.Encoding.PEM,
                                        format=serialization.PublicFormat.SubjectPublicKeyInfo
                                        )
        
        payload = sign_message(sign_key, public_key_pem)
        conn.send(payload)      
        
        payload = conn.recv()    
        if verify_signature(ver_key, payload['sig'], payload['msg']) and payload['msg'] != b"0":
            g_y = load_pem_public_key(payload['msg'], backend=default_backend())
            # Generate shared secret
            s_key = priv_key.exchange(ec.ECDH(), g_y)[:16]
            
            content = 'ola'.encode("ascii")
            # content = input("Enter a message to send to Receiver (to close the connection send 0): ")

            # Set tweak (nonce) to current time since epoch in seconds + 4 byte random value
            tweak = str(time.time()).encode("ascii")[-4:]+os.urandom(4)

            payload = encrypt_message(content, tweak, s_key)

            conn.send(payload)
        else:
            if payload['msg'] != b"0":
                sign = sign_message(sign_key, b"0")
                conn.send(sign)
            else:
                print("A verificação na troca de chaves falhou!")
        conn.close()
    except Exception as e:
        print("Emitter:" + e)
        print("Emitter: Error")

## Receiver

O Receiver, após a troca de chaves, e a cada mensagem recebida, efetua uma verificação da autenticidade e integridade da mesma. 

Inicialmente verifica a validade da assinatura da mensagem, e posteriormente recorre à função auxiliar `decipher_message`, que recebe como parâmetros a chave acordada (`s_key`), o do *tweak_iv/* $\nu$ (`tweak_iv`), o criptograma (`ctxt`) e a tag (`tag`), que deve ser introduzida no momento de decifragem dos dados e, caso não corresponda ao valor esperado, resulta numa exceção.

In [48]:
def Receiver(conn):
    try:
        # Load long-term keys
        sign_key_dump = open("receiver_priv.pem", "rb").read()
        sign_key = load_pem_private_key(sign_key_dump, "receiver".encode("ascii"), backend = default_backend())
        
        ver_key_dump = open("emitter_pub.pem", "rb").read()
        ver_key = load_pem_public_key(ver_key_dump, backend = default_backend())

        payload = conn.recv()        
        if verify_signature(ver_key, payload['sig'], payload['msg']):
            # Session keys
            priv_key = ec.generate_private_key(ec.SECP521R1(), default_backend())
            public_key_pem = priv_key.public_key().public_bytes(
                                                encoding=serialization.Encoding.PEM,
                                                format=serialization.PublicFormat.SubjectPublicKeyInfo
                                                )
            g_x = load_pem_public_key(payload['msg'], backend=default_backend())

            payload = sign_message(sign_key, public_key_pem) 
            conn.send(payload) 

            s_key = priv_key.exchange(ec.ECDH(),g_x)[:16]
            payload = conn.recv()
            
            if payload != b"0":
                ctxt = payload['content']
                tag = payload['tag']
                tweak_iv = payload['tweak_iv']

                msg = decipher_message(s_key, tweak_iv, ctxt, tag)

                if msg:
                    conn.send(msg)
                    print("Received: " + msg.decode("utf-8"))
                else:
                    print("Verificação da mensagem recebida falhou!")
            else:
                print("A verificação na troca de chaves falhou!")
        else:
            response = sign_message(sign_key,b"0")
            conn.send(response) 
        conn.close()
    except Exception as e:
        print("Receiver:" + e)
        print("Receiver: Erro na decifragem")

O estabelecimento de uma conexão entre os dois intervenientes (`Emitter` e `Receiver`)  é feita invocando o método `auto` da classe `Connection` que cria uma comunicação síncrona entre ambos. 
Este sincronismo é exigido pelo protocolo de troca de chaves (Diffie Hellman)

In [49]:
Connection(Emitter, Receiver, timeout=10).auto()

Received: ola             


## Observações

Uma das diferenças notórias entre as versões com e sem curvas elítpicas está no tempo de execução. Esta é uma das vantagens apresentadas pelas curvas elípticas, maior *performance* para um nível de segurança semelhante. Adicionalmente a curvas elípticas possibilitam o uso de chaves de dimensões inferiores face às restantes implementações, para um dado nível de segurança, tornando-as indispensáveis quando o sistema em questão possui recursos reduzidos.